In [23]:
import os
import pandas as pd
import numpy as np
import pandas as pd
import glob

In [9]:
file_directory = "/Users/samxie/Research/CrowdImageComp/99design/OngoingCrawl/Image_99Design/Reg/Reg_Final/CSV/ContestID/"
file_list = [os.path.join(file_directory, file) for file in os.listdir(file_directory) if file.endswith('.csv')]
df_list = []

# loop for reading
for file in file_list:
    temp_df = pd.read_csv(file, dtype = str)
    df_list.append(temp_df)
# combine
ContestFinal_temp = pd.concat(df_list, ignore_index=True)

del temp_df
del file_list
del df_list

print(ContestFinal_temp)

      ContestID                                         ContestURL  \
0       1302846  https://99designs.hk/logo-design/contests/re-d...   
1       1302842  https://99designs.hk/logo-design/contests/desi...   
2       1302831  https://99designs.hk/logo-design/contests/top-...   
3       1302830  https://99designs.hk/logo-design/contests/desi...   
4       1302829  https://99designs.hk/logo-design/contests/auto...   
...         ...                                                ...   
13256   1297279  https://99designs.hk/logo-design/contests/desi...   
13257   1297276  https://99designs.hk/logo-design/contests/retr...   
13258   1297274  https://99designs.hk/logo-design/contests/smil...   
13259   1297273  https://99designs.hk/logo-design/contests/crea...   
13260   1297256  https://99designs.hk/logo-design/contests/glov...   

                                             ContestName   Reward Blind  \
0      HoneyPot THC — Re-design Logo needed for popul...   US$299     0   
1      Ec

In [12]:
ContestAll = (
    ContestFinal_temp.assign(
        ContestID = pd.to_numeric(ContestFinal_temp['ContestID'], errors = 'coerce'),
        CrawlTime = pd.to_datetime(ContestFinal_temp['CrawlTime'], errors = 'coerce')
    )
    .sort_values(by=['ContestID','CrawlTime'], ascending=[False, False])
    .drop_duplicates(subset='ContestID',keep='first')
    # drop CrawlTime Data
    .query("CrawlTime<'2024-11-19'")
    .sort_values(by='ContestID', ascending=False)
)

print(ContestAll)

       ContestID                                         ContestURL  \
11186    1302966  https://99designs.hk/logo-design/contests/pain...   
11197    1302948  https://99designs.hk/logo-design/contests/logo...   
11199    1302942  https://99designs.hk/logo-design/contests/hill...   
8681     1302896  https://99designs.hk/logo-design/contests/inco...   
11223    1302870  https://99designs.hk/logo-design/contests/frie...   
...          ...                                                ...   
11537    1297059  https://99designs.hk/logo-design/contests/mini...   
3593     1296771  https://99designs.hk/logo-design/contests/fres...   
10641    1293522  https://99designs.hk/logo-design/contests/inno...   
7756     1283670  https://99designs.hk/logo-design/contests/lake...   
7300     1277328  https://99designs.hk/logo-design/contests/sand...   

                                             ContestName   Reward Blind  \
11186    Crotti’s Painting — Painting company funny logo   US$899     0 

## Simi Loading

In [18]:
Simi_files = [
    '/Users/samxie/Research/CrowdImageComp/99design/OngoingCrawl/Image_99Design/Reg/Reg_Final/CSV/Ongoing/Sim_All_Contest1103_1.csv',
    '/Users/samxie/Research/CrowdImageComp/99design/OngoingCrawl/Image_99Design/Reg/Reg_Final/CSV/Ongoing/Sim_All_Contest1103_2.csv',
    '/Users/samxie/Research/CrowdImageComp/99design/OngoingCrawl/Image_99Design/Reg/Reg_Final/CSV/Ongoing/Sim_All_Contest1103_3.csv',
    '/Users/samxie/Research/CrowdImageComp/99design/OngoingCrawl/Image_99Design/Reg/Reg_Final/CSV/Ongoing/SimiContestFinal.csv'
]

Simi_All = pd.concat(
    [
        pd.read_csv(file, dtype=str).apply(pd.to_numeric, errors='coerce')
        for file in Simi_files
    ],
    ignore_index = True
)

Simi_All = (
    Simi_All
    .groupby('ContestID', as_index=False)
    .apply(lambda group: group.assign(
        EntryNumCollected_Simi=len(group),
        EntryLast_Simi = group['Entry'].max(),
        CollectedMissing_Simi = np.abs(len(group)-group['Entry'].max()),
        CollectedBiased_Simi = np.abs(len(group)-group['Entry'].max())/group['Entry'].max(),
        CallBack_Simi = 1 - (np.abs(len(group)-group['Entry'].max())/group['Entry'].max())
        )
    )
)

Simi_All = (
    Simi_All
    .groupby(['ContestID','Entry'], as_index=False)
    .first()
)

Simi_All = Simi_All.sort_values(by=['ContestID','Entry'], ascending=[False, True])

print(Simi_All)

        ContestID  Entry  Before_Avg_Sim_VGG  After_Avg_Sim_VGG  Ref_Max_VGG  \
114207    1302966      1              0.0000             0.8509       0.8886   
114208    1302966      2              0.9131             0.8909       0.9125   
114209    1302966      3              0.8934             0.8551       0.8719   
114210    1302966      4              0.8641             0.8277       0.8484   
114211    1302966      5              0.8028             0.7980       0.8225   
...           ...    ...                 ...                ...          ...   
80        1277328    211              0.9068             0.9718       0.8860   
81        1277328    212              0.9051             0.9717       0.8748   
82        1277328    213              0.8968             0.9808       0.8691   
83        1277328    214              0.9050             0.9770       0.8760   
84        1277328    215              0.8972             0.0000       0.8626   

        Ref_Max_3_VGG  Ref_Avg_VGG  Bef

/var/folders/x9/p1d98gj518326zdc2s5gzn8w0000gn/T/ipykernel_35561/2614596495.py:19: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda group: group.assign(


## Crawl Ideas 

In [31]:
# Define the directory containing CSV files
csv_directory = "/Users/samxie/Research/CrowdImageComp/99design/OngoingCrawl/Image_99Design/Reg/Reg_Final/CSV/Ongoing/SubmissionLevel"
csv_files = glob.glob(f"{csv_directory}/*.csv")

# Read all CSV files in the directory and merge them
Ideas_RAW = pd.concat(
    [pd.read_csv(file, dtype='object') for file in csv_files],
    ignore_index=True
)

Seeker_Info = (
    pd.read_csv("/Users/samxie/Research/CrowdImageComp/99design/OngoingCrawl/Image_99Design/Reg/Reg_Final/CSV/Ongoing/Seeker_Final.csv")
    .groupby('ContestID', as_index=False)
    .first()
)

print(Seeker_Info)

      ContestID                                        ContestName  \
0       1277328      longview deli — Sandwich Deli New Fresh Local   
1       1283670  Lake County Barbecue and Wine Festival — Lake ...   
2       1293522  Jazz Cactus — Innovative cannabis company look...   
3       1296771  Park Shore — We need a "fresh" looking logo fo...   
4       1297059  Medici Mining Investment — Mining company look...   
...         ...                                                ...   
1827    1302870  Friend Funnel — Friend Funnel Logo Needed - Fu...   
1828    1302896  Archie Plante Oilfield Inspection & Consulting...   
1829    1302942     Hillside Sporting — Hillside Sporting New Logo   
1830    1302948    Ski Girl Hot Cocoa — Logo for a Hot Cocoa Stand   
1831    1302966    Crotti’s Painting — Painting company funny logo   

                                             ContestURL             Seeker  \
0     https://99designs.hk/logo-design/contests/sand...          seyam139t   
1  

In [52]:
Seeker_One = (
    Seeker_Info
    .assign(ContestNum=Seeker_Info.groupby('Seeker')['Seeker'].transform('size'))
    .query("ContestNum==1")
    .drop(columns=['ContestNum', 'Solver', 'IdeasNum'])
)
print(Seeker_One)

      ContestID                                        ContestName  \
0       1277328      longview deli — Sandwich Deli New Fresh Local   
1       1283670  Lake County Barbecue and Wine Festival — Lake ...   
2       1293522  Jazz Cactus — Innovative cannabis company look...   
3       1296771  Park Shore — We need a "fresh" looking logo fo...   
4       1297059  Medici Mining Investment — Mining company look...   
...         ...                                                ...   
1826    1302868  GlidePath Tours — design a sophisticated yet f...   
1828    1302896  Archie Plante Oilfield Inspection & Consulting...   
1829    1302942     Hillside Sporting — Hillside Sporting New Logo   
1830    1302948    Ski Girl Hot Cocoa — Logo for a Hot Cocoa Stand   
1831    1302966    Crotti’s Painting — Painting company funny logo   

                                             ContestURL             Seeker  
0     https://99designs.hk/logo-design/contests/sand...          seyam139t  
1    

In [44]:
winner_file = "/Users/samxie/Research/CrowdImageComp/99design/OngoingCrawl/Image_99Design/Reg/Reg_Final/CSV/Ongoing/Winner_Rating_Final1222.csv"

Winner_Info = (
    pd.read_csv(winner_file, dtype='object')
    .sort_values(by=['ContestID', 'Entry', 'WinnerCheck'], ascending=[False, True, False])  
    .drop_duplicates(subset=['ContestID', 'Entry'], keep='first')  
    .sort_values(by=['ContestID', 'Entry'], ascending=[False, True]) 
    [['ContestID', 'Entry', 'WinnerCheck', 'RatingCheck']]  
)

Ref_Info = (
    pd.read_csv(winner_file, dtype = 'object')
    .sort_values(by=['ContestID'], ascending=False)
    .drop_duplicates(subset=['ContestID'], keep='first')[['ContestID','InspirationNum','RefNum']]
)

print(Winner_Info)
print('------------------')
print(Ref_Info)


      ContestID Entry WinnerCheck RatingCheck
0       1302948    21           1           5
3       1302942     1           0           4
1       1302942     2           1           5
2       1302942     6           1         NaN
4       1302896    12           1         NaN
...         ...   ...         ...         ...
13378   1296771   267           1         NaN
13379   1296771   296           0           5
13383   1293522   130           0           5
13384   1293522   131           0           1
13382   1293522   155           1         NaN

[13385 rows x 4 columns]
------------------
      ContestID InspirationNum RefNum
0       1302948              8      0
2       1302942             10      1
4       1302896              2      1
128     1302870              1      1
184     1302868             10      1
...         ...            ...    ...
13338   1297936              5      1
13367   1297885              6      0
13377   1297059              5      1
13378   1296771        

In [56]:
#  Join
Seeker_One['ContestID'] = Seeker_One['ContestID'].astype(str)
Ideas_RAW['ContestID'] = Ideas_RAW['ContestID'].astype(str)
Winner_Info['ContestID'] = Winner_Info['ContestID'].astype(str)
Winner_Info['Entry'] = Winner_Info['Entry'].astype(str)

Ref_Info['ContestID'] = Ref_Info['ContestID'].astype(str)
Ideas_Process = pd.merge(Seeker_One, Ideas_RAW, how='left', on='ContestID')
print(Ideas_Process.shape[0])
print('--------------------')
Ideas_Process = pd.merge(Ideas_Process, Winner_Info, how='left', on=['ContestID','Entry'])
print(Ideas_Process.shape[0])
print('--------------------')
Ideas_Process = pd.merge(Ideas_Process, Ref_Info, how = 'left', on='ContestID')
print(Ideas_Process.shape[0])
print('--------------------')

# PackageLevel not NA
Ideas_Process = Ideas_Process[Ideas_Process['PackageLevel'].notna()]

Ideas_Process = (
    Ideas_Process
    .sort_values(by=['ContestID','Entry','Winner','Rating'], ascending=[False,True,False,False])
    .groupby(['ContestID','Entry'], as_index=False)
    .first()
    .sort_values(by=['ContestID','Entry','Winner','Rating'], ascending=[False,True,False,False])
)

Ideas_Process = Ideas_Process.sort_values(by=['ContestID', 'Entry', 'Winner', 'Rating'], ascending=[False, True, False, False])

Ideas_Process = (
    Ideas_Process
    .groupby('ContestID', as_index = False)
    .apply(lambda group: group.assign(
        Winner=np.where(group['WinnerCheck'].notna(), group['WinnerCheck'], group['Winner']),
        Rating=np.where(group['RatingCheck'].notna(), group['RatingCheck'], group['Rating']),
        HasWinner=group['Winner'].max(),
        EntryNumCollected_Ideas=len(group),
        EntryLast_Ideas=group['Entry'].astype(float).max(),
        CollectedMissing_Ideas=np.abs(len(group) - group['Entry'].astype(float).max()),
        CollectedBiased_Ideas=np.abs(len(group) - group['Entry'].astype(float).max()) / group['Entry'].astype(float).max(),
        CallBack_Ideas=1 - (np.abs(len(group) - group['Entry'].astype(float).max()) / group['Entry'].astype(float).max())       
    ))
    .reset_index(drop=True)
)

Ideas_Process = Ideas_Process.drop(columns=['WinnerEntry', 'WinnerCheck', 'RatingCheck'])

Ideas_Process = Ideas_Process[Ideas_Process['InspirationNum'].notna()]
print(Ideas_Process.shape[0])
print('--------------------')

315639
--------------------
315639
--------------------
315639
--------------------


/var/folders/x9/p1d98gj518326zdc2s5gzn8w0000gn/T/ipykernel_35561/3192263427.py:34: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  .apply(lambda group: group.assign(


90343
--------------------
